## Goal:
- To generate and retrive enough information from DataBase


## Description of the data

**books:**
Contains data on books:

- `book_id`

- `author_id`
-`title`

- `num_pages` — number of pages

- `publication_date`

- `publisher_id

***authors:***
Contains data on authors:

- `author_id`
- `author`

**publishers:**
Contains data on publishers:
- `publisher_id`
- `publisher`

**ratings:**
Contains data on user ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

**reviews:**
Contains data on customer reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book
- `text` — the text of the review


## Accessing the database Connect to the database



In [1]:
# Loading libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Colecting data

In [3]:
books = '''
        SELECT * FROM BOOKS
        '''

books = pd.io.sql.read_sql(books, con = engine)     

In [4]:
authors = '''
        SELECT * FROM authors

        '''
authors = pd.io.sql.read_sql(authors, con = engine)

In [5]:
publishers =  '''
        SELECT * FROM publishers

        '''
publishers = pd.io.sql.read_sql(publishers, con = engine)

In [6]:
ratings =  '''
        SELECT * FROM ratings

        '''
ratings = pd.io.sql.read_sql(ratings, con = engine)

In [7]:
reviews =  '''
        SELECT * FROM reviews

        '''
reviews = pd.io.sql.read_sql(reviews, con = engine)

## Data Preprocessing 
1. Study the tables (print the first rows)
2. Write SQL script inside pandas read_sql method as follows:

In [8]:
# Print sample of the data
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
# Check missing values
books.isna().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

In [10]:
# Check for dublicates
display(books.duplicated().sum()) 

0

In [11]:
# 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [12]:
# Check missing values
authors.isna().sum()

author_id    0
author       0
dtype: int64

In [13]:
# Check for dublicates
display(authors.duplicated().sum())

0

In [14]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [15]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [16]:
# Check missing values
ratings.isna().sum()

rating_id    0
book_id      0
username     0
rating       0
dtype: int64

In [17]:
# Check for dublicates
display(ratings.duplicated().sum())

0

In [18]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [19]:
# Check missing values
reviews.isna().sum()

review_id    0
book_id      0
username     0
text         0
dtype: int64

In [20]:
#Check for dublicates
display(reviews.duplicated().sum())

0

### Conclusion
- There is no missing values and dublicates in dataset

In [21]:
# Let's create a function that displays the result of an SQL query:
def sql_result(query):
    return pd.io.sql.read_sql(query, con = engine)

In [22]:
# Check if it works
query = \
'''

SELECT * 
FROM books
LIMIT 5

'''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Analysis
 Now we will analyze the collected data and compute some statistics.

### Find the number of books released after January 1, 2000

In [23]:
query_1 =   '''
                    SELECT
                        COUNT(book_id) AS book_cnt
                    FROM
                        books
                    WHERE  
                        publication_date::date > '2000-01-01'
                    ''' 
query_1 = pd.io.sql.read_sql(query_1, con = engine)        
display(query_1)

,book_cnt
0,819


- There are 819 books released after 2000

### Find the number of user reviews and the average rating for each book.

In [24]:
query = \
'''

SELECT
  books.title,
  COUNT (distinct review_id) as review_cnt,
  AVG(rating) AS avg_rating
FROM
  books
LEFT JOIN
  reviews ON reviews.book_id = books.book_id
LEFT JOIN
  ratings ON ratings.book_id = books.book_id
GROUP BY
  books.title
ORDER BY 
    COUNT(reviews.review_id) DESC

'''
sql_result(query)

,title,review_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Natural Way to Draw,0,3.000000


- For each title of  book we see the number of review and the average rating.

### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [25]:
query_3 = '''
        SELECT
            publishers.publisher_id AS publisher_id,
            publishers.publisher AS publisher_name,
            COUNT(DISTINCT(books.book_id)) AS cnt_books
        FROM
            publishers
        INNER JOIN books ON books.publisher_id = publishers.publisher_id
        WHERE
            books.num_pages > 50
        GROUP BY
            publishers.publisher_id, publishers.publisher
        ORDER BY
            cnt_books DESC
        LIMIT 20;
            
    
'''
query_3 = pd.io.sql.read_sql(query_3, con = engine)        
display(query_3)

,publisher_id,publisher_name,cnt_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
5,33,Ballantine Books,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,333,William Morrow Paperbacks,13


- The top 20 of publishers are displayed in this table
- Penguin Books released the highest number of books.

### Identify the author with the highest average book rating: look only at books with at least 50 ratings

In [26]:
query_4 = '''
    SELECT    
        books.author_id AS author_id,
        authors.author AS author_name,
        ratings.book_id AS book_id,
        AVG(ratings.rating) AS avg_reating    
    FROM
        ratings
    INNER JOIN
        books ON books.book_id = ratings.book_id
    INNER JOIN
        authors ON authors.author_id = books.author_id    
    GROUP BY
        ratings.book_id, books.author_id, authors.author
    HAVING
        COUNT(ratings.rating_id) > 49
    ORDER BY
        avg_reating DESC
    LIMIT 20;

'''
query_4 = pd.io.sql.read_sql(query_4, con = engine)        
display(query_4)

,author_id,author_name,book_id,avg_reating
0,236,J.K. Rowling/Mary GrandPré,302,4.414634
1,240,J.R.R. Tolkien,722,4.391892
2,236,J.K. Rowling/Mary GrandPré,299,4.287500
3,402,Markus Zusak/Cao Xuân Việt Khương,656,4.264151
4,236,J.K. Rowling/Mary GrandPré,300,4.246575
5,376,Louisa May Alcott,399,4.192308
6,236,J.K. Rowling/Mary GrandPré,301,4.186667
7,240,J.R.R. Tolkien,750,4.125000
8,498,Rick Riordan,779,4.080645
9,621,William Golding,405,3.901408


- We see the top 20 outhors . The highest average rating is 4.41.

### Find the average number of text reviews among users who rated more than 50 books.

In [27]:
query = \
'''
SELECT
  AVG(review_cnt) as avg_review_cnt
FROM
(SELECT
  COUNT(review_id) as review_cnt
FROM
  reviews
WHERE username IN
(SELECT
  username
FROM
  ratings
GROUP BY
  username
HAVING
  COUNT(ratings) > 50)
GROUP BY
  username) as sub_review_cnt 

'''
sql_result(query)

,avg_review_cnt
0,24.333333



- The average number of text review is 24.333333

## Conclusion
We went through the topics that were important for us to check out
1. Number of books released after January 1, 2000 are 819 books
2. For each book we retrive title, avg rating and counted rating (see N7.2)
3. The publisher that has released the greatest number of books with more than 50 pages:
- `publisher_id`is 212	 and  Penguin Books released the highest number of books:42
3. The author with the highest average book rating is J.K. Rowling/Mary GrandPré with rating 4.41
4. The average number of text reviews is 24.33333

- This definitely brought us important initial information that will help us in the further process of developing a new product.